In [1]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict
from langchain_openai import OpenAI
from dotenv import load_dotenv

In [2]:
load_dotenv()
model = OpenAI(temperature=0.7)

In [3]:
class BlogState(TypedDict):
    title: str
    outline: str
    content: str

In [4]:
def create_outline(state: BlogState) -> BlogState:
    #fetch title from state
    title = state['title']
    #call llm gen outline
    prompt = f"Create a detailed outline for a blog post titled: {title}"
    outline = model.invoke(prompt).content
    #update outline in state
    state['outline'] = outline
    return state

In [6]:
def create_blog(state: BlogState) -> BlogState:
    #fetch outline from state
    title = state['title']
    outline = state['outline']
    prompt = f"Write a comprehensive blog post based on the title: {title} and the outline: {outline} \n {outline}"
    content = model.invoke(prompt).content
    state['content'] = content
    return state 

In [10]:
graph = StateGraph(BlogState)

# nodes 
graph.add_node("create_outline",create_outline)
graph.add_node("create_blog",create_blog)
#edges
graph.add_edge(START,"create_outline")
graph.add_edge("create_outline","create_blog")
graph.add_edge("create_blog",END)

workflow = graph.compile()

In [ ]:
intial_state = {"title":"rise of AI in India"}
final_state =workflow.invoke(intial_state)
print(final_state)

In [ ]:
print(final_state['content'])